In [13]:
from datasets import load_dataset
from transformers import AutoTokenizer
import csv

In [15]:
# Create questions csv
category = "anatomy"
ds = load_dataset("edinburgh-dawg/mmlu-redux", category)
with open('questions.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['question_idx', 'question', 'answer_choices', 'prompt', 'correct_answer', 'category'])
    for i, ex in enumerate(ds['test']):
        prompt = ex['question'] + "\n\n" + "Answer Choices:\n" + "\n".join(ex['choices'])
        writer.writerow([i, ex['question'], ex['choices'], prompt, ex['answer'], category])

In [14]:
deepseekr1_0528_tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-0528")

In [ ]:
# Assuming DeepSeek-R1-0528_mmlu-redux_results.csv exists with the following columns:
# question_idx, question, answer_choices, full_prompt, correct_answer, full_cot, predicted_answer, category

early_decoders = [
    "probe_model_answer_correct", "probe_model_answer", "probe_correct_answer",
    "observer_model_answer_correct", "observer_model_answer", "observer_correct_answer",
]

def generate_fake_data(early_decoder):
    if early_decoder.endswith('model_answer_correct'): # did the model answer correctly?
        return np.random.dirichlet(np.ones(2)).tolist()
    elif early_decoder.endswith('model_answer'): # what did the model answer?
        return np.random.dirichlet(np.ones(4)).tolist() 
    elif early_decoder.endswith('correct_answer'): # what was the correct answer?
        return np.random.dirichlet(np.ones(4)).tolist()

questions_df = pd.read_csv('DeepSeek-R1-0528_mmlu-redux_results.csv')
for row in questions_df.iterrows():
    question_idx = row['question_idx']
    question = row['question']
    answer_choices = row['answer_choices']
    prompt = row['full_prompt']
    correct_answer = row['correct_answer']
    full_cot = row['full_cot']
    predicted_answer = row['predicted_answer']
    category = row['category']

    for early_decoder in early_decoders:
        full_cot_tokens = tokenizer.tokenize(full_cot)
        for i, token in enumerate(full_cot_tokens):
            token_text = tokenizer.convert_ids_to_tokens(token)
            probe_output = generate_fake_data(early_decoder)
            writer.writerow([question_idx, token_idx, token_text, early_decoder, probe_output])